In [ ]:
import pickle

import holoviews as hv
import hvplot.pandas
import jax.numpy as jnp
import numpy as np
import pandas as pd
import panel as pn
import param

In [ ]:
%load_ext autoreload
%autoreload 2
import opticalmaterials
import spectra
import util

In [ ]:
hv.extension("bokeh")

# Spectral response

In [ ]:
bins = np.arange(300, 1000).astype(float)

In [ ]:
%%time
with open("compoundprisms/rii.pickle", "rb") as f:
    catalog = pickle.load(f)

In [ ]:
def delta(n, gamma=np.deg2rad(30), theta0=0, beta=0):
    alpha = gamma - beta
    # for theta0=0, beta=0, simplifies to:
    # jnp.arcsin(n(lmbda)*jnp.sin(gamma)) - gamma
    return (
        theta0
        - gamma
        - jnp.arcsin(n * jnp.sin(jnp.arcsin(jnp.sin(theta0 - beta) / n) - alpha))
    )

In [ ]:
n, dn = opticalmaterials.get_n(catalog[("BK7", "SCHOTT")], nanometers=True)

In [ ]:
# Dora calculated alpha angles using 400-800nm wavelength range (11deg 21' -> 0.2372deg angle diff)

In [ ]:
# desired range: 550nm to 665nm

In [ ]:
gamma = np.deg2rad(11 + 21 / 60)
np.rad2deg(delta(n(400), gamma=gamma) - delta(n(800), gamma=gamma))

In [ ]:
gamma = np.deg2rad(11 + 21 / 60)
gamma = np.deg2rad(18 + 9 / 60)
np.rad2deg(delta(n(550), gamma=gamma) - delta(n(665), gamma=gamma))

In [ ]:
hv.Curve((bins, np.rad2deg(delta(n(bins), gamma=np.deg2rad(8)))))

In [ ]:
hv.Curve((bins, n(bins)))

In [ ]:
hv.Curve((bins, dn(bins))) * hv.Curve((bins[:-1], np.diff(n(bins))))

In [ ]:
def evaluate_prisms(glasses, wavelength_min=550, wavelength_max=665):
    d = {}
    for glass_name, glass in glasses.items():
        gamma = np.deg2rad(18 + 9 / 60)
        try:
            n, _ = opticalmaterials.get_n(glass, nanometers=True)
            n_blue = n(wavelength_min)
            n_red = n(wavelength_max)
            angle = np.rad2deg(delta(n_blue, gamma=gamma) - delta(n_red, gamma=gamma))
            d[glass_name] = (
                pd.Series(
                    {
                        "n_blue": n_blue,
                        "n_red": n_red,
                        "n_diff": n_blue - n_red,
                        "angle": angle,
                    }
                )
                .to_frame()
                .T
            )
        except:
            print(f"skipping {glass_name}")
    # for filter_combo in filter_combo_names:
    #     dc = dichroics[filter_combo[0]]
    #     lp = longpass_filters[filter_combo[1]]
    #     dd = {}
    #     for fp_name in fp_names:
    #         ex = fps[fp_name]["spectra"]["ex"]
    #         em = fps[fp_name]["spectra"]["em"]
    #         dd[fp_name] = (
    #             fp_efficiency(
    #                 dc,
    #                 lp,
    #                 ex,
    #                 em,
    #                 filter_combinations.loc[filter_combo]["ex_cutoff"],
    #                 filter_combinations.loc[filter_combo]["em_cutoff"],
    #             )
    #             .to_frame()
    #             .T
    #         )
    #     d[filter_combo] = (
    #         pd.concat(dd, axis=1)
    #         .reorder_levels([1, 2, 0], axis=1)
    #         .sort_index(axis=1, level=1, sort_remaining=False)
    #     )
    return pd.concat(d, axis=0).droplevel(-1)

In [ ]:
catalog_subset = {
    k: v
    for k, v in catalog.items()
    if any(
        c in k[0] or c in k[1]
        for c in ("SCHOTT", "CDGM", "HIKARI", "OHARA", "SUMITA", "HOYA")
    )
}

In [ ]:
%%time
p = evaluate_prisms(catalog_subset)

In [ ]:
p

In [ ]:
p[~p["angle"].isnull()].sort_values("n_blue")

In [ ]:
p.hvplot.scatter("n_blue", "n_red")

In [ ]:
# UPDATE GH